Convert segmentation masks to GeoTIFF.

In [1]:
import copy
import glob
import os

import geopandas as gpd
import numpy as np
import rasterio.mask
import rasterio.plot

In [2]:
# Load the masks.
masks = sorted(glob.glob("C:/Users/Dimit/Downloads/RoofSense.v3-pre-test_all-classes.png-mask-semantic/train/*.png"))
masks

['C:/Users/Dimit/Downloads/RoofSense.v3-pre-test_all-classes.png-mask-semantic/train\\chips-119_png.rf.413d2b3bfb0c13f076956d7933d22c34_mask.png',
 'C:/Users/Dimit/Downloads/RoofSense.v3-pre-test_all-classes.png-mask-semantic/train\\chips-120_png.rf.7e334b4f1537af3f8b64a01b29ec75f3_mask.png']

In [3]:
# Match the masks to the corresponding stack chips.
chips = [os.path.basename(mask[:mask.rfind(".rf")]).replace("-", ".").rstrip("_png") for mask in masks]
chips

['chips.119', 'chips.120']

In [8]:
# Load and dissolve the corresponding roof surfaces.
surfs = gpd.read_file("data/tmp/9-284-556.surfs.gpkg").dissolve()

# Compute the inverse class frequencies to use as training weights.
weights = np.zeros(10, dtype=np.int64)

chip_src: rasterio.io.DatasetReader
mask_src: rasterio.io.DatasetReader
mask_dst: rasterio.io.DatasetWriter
for chip, mask in zip(chips, masks):
    with rasterio.open(os.path.join("data/imgs/tif", f"{chip}.tif")) as chip_src, rasterio.open(mask) as mask_src:
        mask_data = mask_src.read()

        mask_meta = copy.deepcopy(chip_src.meta)
        mask_meta.update(count=1, dtype=np.uint8)

        with rasterio.open(os.path.join("data/msks/", f"{chip}.tif"), mode="w+", **mask_meta) as mask_dst:
            # Georeference the masks.
            mask_dst.write(mask_data)

            # Remask the background.
            mask_data = rasterio.mask.mask(mask_dst, shapes=surfs.geometry)[0]
            mask_dst.write(mask_data)

            # Mark the remaining unlabeled pixels as background
            # TODO: Check whether this approach is correct..
            #      https://stackoverflow.com/questions/54887933/how-to-to-drop-a-specific-labeled-pixels-in-semantic-segmentation
            # mask_data[mask_data == 0] = 9
            # mask_dst.write(mask_data)

            indices, counts = np.unique(mask_data, return_counts=True)
            weights[indices] += counts

weights

C:\Documents\RoofSense\venv\Lib\site-packages\rasterio\__init__.py:317: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


CPLE_AppDefinedError: Deleting data/msks/chips.119.tif failed: Permission denied

In [5]:
weights = weights / weights.sum()
weights

array([0.75384903, 0.        , 0.07089233, 0.08757973, 0.00534439,
       0.02285194, 0.        , 0.02597427, 0.0335083 , 0.        ])

In [6]:
weights = np.divide(1, weights, where=weights != 0)
weights

array([  1.32652555,   0.        ,  14.10589755,  11.41816756,
       187.11206281,  43.75995326,   0.        ,  38.49963284,
        29.84335155,   0.        ])

In [7]:
# Serialize the weights.
np.save("data/tmp/9-284-556.wght",weights.astype(np.float32))